In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
from astropy import units as u
from astropy.coordinates import SkyCoord, EarthLocation
from astropy.io import fits
from spectral_cube import SpectralCube
from spectral_cube import BooleanArrayMask
from astropy.convolution import Gaussian1DKernel, convolve
import aplpy  
from astropy.wcs import WCS
from reproject import reproject_interp
from astroquery.vizier import Vizier
from astroquery.skyview import SkyView
from astropy.time import Time
import csv
from astropy.io import ascii
import pandas as pd

Vizier.ROW_LIMIT = -1

mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.size'] = '16'
mpl.rcParams['xtick.direction'] = 'in'
mpl.rcParams['ytick.direction'] = 'in'
%matplotlib inline

In [ ]:
def extract_spectra(field, catalogues, sigma):
    # read in ASKAP data cube

    cube_askap = SpectralCube.read('/Users/denes/Research/high_lat_HI/ASKAP_data/data_cubes/{}_MW.fits'.format(field))  # Open the FITS file for reading
    askap = fits.open('/Users/denes/Research/high_lat_HI/ASKAP_data/data_cubes/{}_MW.fits'.format(field))  # Open the FITS file for reading
    d_askap = askap[0].data
    h_askap = askap[0].header
    w_askap = WCS(h_askap, askap)
    #print(h_askap)

    askap_cont = fits.open('/Users/denes/Research/High_lat_HI/ASKAP_data/continuum_data/{}_cont_smooth_regrid.fits'.format(field))  # Open the FITS file for reading
    d_askap_cont = askap_cont[0].data
    h_askap_cont = askap_cont[0].header
    w_askap_cont = WCS(h_askap_cont, askap_cont)

    coordinate = '{} {}'.format(h_askap['CRVAL1'], h_askap['CRVAL2'])
    c = SkyCoord(coordinate, unit=(u.deg, u.deg))
    print(c.ra.hms, c.dec)
    print('pixels HI, cont:', h_askap['NAXIS1'], h_askap['NAXIS2'], h_askap_cont['NAXIS1'], h_askap_cont['NAXIS2'])

    MRO = EarthLocation.of_site('mro') 
    #barycorr = c.radial_velocity_correction(obstime=Time('2019-10-25'), location=MRO) 
    barycorr = c.radial_velocity_correction(obstime=Time(h_askap['DATE-OBS']), location=MRO) 
    lsr_corr = 9*np.cos(c.galactic.l.deg)*np.cos(c.galactic.b.deg)+12*np.sin(c.galactic.l.deg)*np.cos(c.galactic.b.deg)+7*np.sin(c.galactic.b.deg)
    print('LSR corr: ',lsr_corr )
    restfreq = 1.420405752 * u.GHz  # rest frequency of HI
    freq_to_vel = u.doppler_radio(restfreq) # using the optical convention
    vel_askap = (cube_askap.spectral_axis).to(u.km / u.s, equivalencies=freq_to_vel) #+ (-5.875) *(u.km / u.s)

    # read in ASKAP catalogue

    data_1 = pd.read_csv('/Users/denes/Research/high_lat_HI/ASKAP_data/continuum_cat/{}'.format(catalogues[0]))
    data_2 = pd.read_csv('/Users/denes/Research/high_lat_HI/ASKAP_data/continuum_cat/{}'.format(catalogues[1]))

    df_1 = pd.DataFrame(data_1)
    df_2 = pd.DataFrame(data_2)

    # concatenate catalogues
    df = pd.concat([df_1, df_2])

    # do a cut on the catalog

    sources = df[df['flux_peak']>15]
    print(len(sources))

    ra = np.array(sources['ra_hms_cont'])
    dec = np.array(sources['dec_dms_cont'])
    n_pix = np.array(sources['n_pix'])
    f_peak = np.array(sources['flux_peak'])

    # extract spectra
    # filter for S/N
    # make plots

    detection = []
    rms_abs = []
    tau_max = []
    min_askap = []

    for i in range(len(sources)):
        coord = '{} {}'.format(ra[i], dec[i])
        c2 = SkyCoord(coord, unit=(u.hourangle, u.deg))
        l = np.radians(c2.galactic.l.deg)
        b = np.radians(c2.galactic.b.deg)
        lsr_corr = (9*np.cos(l)*np.cos(b)) + (12*np.sin(l)*np.cos(b)) + (7*np.sin(b))
        #print('lsr corr: ', lsr_corr, c2.galactic.l.deg, c2.galactic.b.deg)
        pixels_askap = c2.to_pixel(w_askap)
        pixels_askap_cont = c2.to_pixel(w_askap_cont)
        #print(coord )
        #print(pixels_askap)

        spectrum_askap = cube_askap[:, int(pixels_askap[1]), int(pixels_askap[0])]  # 10:09:10 -28:55:57
        continuum_askap = d_askap_cont[int(pixels_askap_cont[1]), int(pixels_askap_cont[0])]
        #print('continuum peak: ', continuum_askap*1000, f_peak[i])
        spectrum_askap_cor = (spectrum_askap.value+continuum_askap)/continuum_askap
        #spectrum_askap_cor = (spectrum_askap.value+(f_peak[i]/1000))/(f_peak[i]/1000)
        tau_askap = np.log((spectrum_askap.value+continuum_askap)/continuum_askap) * -1.
        rms = np.sqrt(np.mean(spectrum_askap.value**2))
        rms_abs.append(rms)
        min_askap.append(abs(np.min(spectrum_askap.value)))
        tau_max.append(np.max(tau_askap))

        if abs(np.min(spectrum_askap.value)) > rms*sigma:
            detection.append(i)

            # plot
            fig = plt.figure(figsize=(16, 12))
            ax = fig.add_subplot(111)
            ax.set_title('{}{}'.format(ra[i], dec[i]), fontsize=20)
            plt.grid(linestyle=':', color='grey')

            #plt.plot(vel_askap, (spectrum_askap.value+(f_peak[i]/1000))/(f_peak[i]/1000), 'C1', linewidth=3, label='ASKAP')
            plt.plot(vel_askap, (spectrum_askap.value+continuum_askap)/continuum_askap, 'C1', linewidth=3, label='ASKAP')
            plt.plot(vel_askap, spectrum_askap.value+1, 'C2', linewidth=3, label='ASKAP')
            ax.axhspan(1-rms, 1+rms, alpha=0.5, color='lightgrey')
            #plt.title('102809-264418', fontsize=30)
            plt.ylabel(r'e$^{-\tau}$', fontsize=28)
            plt.xlabel("v [km/s]", fontsize=28)
            plt.xlim(-300,600)
            #plt.ylim(0.92,1.02)
            plt.axhline(1, color='k', linestyle='--')
            #plt.text(-200, 0.9, continuum_askap, fontsize=20)
            plt.legend(fontsize=20)
            ax.tick_params(axis='both', which='major', labelsize=25)
            fig.savefig('./spectra_plots/{}/{}{}_ASKAP_spectra_test.png'.format(field, ra[i].replace(':',''), dec[i].replace(':','')))

            df_spectra = pd.DataFrame()
            df_spectra['velocity'] = vel_askap + lsr_corr * (u.km / u.s)
            df_spectra['Amplitude'] = (spectrum_askap.value+continuum_askap)/continuum_askap
            df_spectra.to_csv('./spectra/{}/{}{}_askap_spectrum.txt'.format(field, ra[i].replace(':',''), dec[i].replace(':','')), sep='\t')


    sources['rms_abs'] = rms_abs   
    sources['tau_max'] = tau_max
    sources['min_askap'] = min_askap
    print('{} sigma detections:'.format(sigma), len(detection))    

    sources_good_5 = sources[sources['min_askap'] > sources['rms_abs']*sigma]
    sources_good_5.to_csv('./{}_sigma_askap_sources_{}.csv'.format(sigma, field))

In [ ]:
#extract_spectra('Hydra', ['AS102_Continuum_Island_Catalogue_10609_119.csv', 'AS102_Continuum_Island_Catalogue_10612_123.csv'], 5)
#extract_spectra('Norma', ['AS102_Continuum_Island_Catalogue_11832_3718.csv', 'AS102_Continuum_Island_Catalogue_12209_3734.csv'], 5)
extract_spectra('Norma', ['AS102_Continuum_Island_Catalogue_11816_3700.csv', 'AS102_Continuum_Island_Catalogue_12209_3734.csv'], 5)

#extract_spectra('NGC4636', ['AS102_Continuum_Island_Catalogue_10809_2351.csv', 'AS102_Continuum_Island_Catalogue_10736_3690.csv'], 5)


